In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import pickle
nltk.download("punkt")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from gensim.models import Word2Vec, KeyedVectors

[nltk_data] Downloading package punkt to C:\Users\Malathi
[nltk_data]     M\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#!pip install tensorflow tensorflow-addons seqeval


ERROR: Could not find a version that satisfies the requirement tensorflow-addons (from versions: none)
ERROR: No matching distribution found for tensorflow-addons


Load data

In [2]:
df = pd.read_csv("C:/Users/Malathi M/OneDrive/Documents/MDTE25/guvi final project/Main project/news.tsv.zip", sep="\t")
print(df.head())



  News ID Category         Topic  \
0  N10000   sports        soccer   
1  N10001     news  newspolitics   
2  N10002     news        newsus   
3  N10003     news  newspolitics   
4  N10004     news     newsworld   

                                            Headline  \
0  Predicting Atlanta United's lineup against Col...   
1  Mitch McConnell: DC statehood push is 'full bo...   
2            Home In North Highlands Damaged By Fire   
3  Meghan McCain blames 'liberal media' and 'thir...   
4                            Today in History: Aug 1   

                                           News body  \
0  Only FIVE internationals allowed, count em, FI...   
1  WASHINGTON -- Senate Majority Leader Mitch McC...   
2  NORTH HIGHLANDS (CBS13)   Fire damaged a home ...   
3  Meghan McCain is speaking out after a journali...   
4  1714: George I becomes King Georg Ludwig, Elec...   

                                Title entity  \
0  {"Atlanta United's": 'Atlanta United FC'}   
1            

In [4]:
df["Category"].value_counts()

Category
sports           30557
news             26689
finance          10571
lifestyle         7405
autos             5494
travel            5381
foodanddrink      5286
video             4968
tv                3981
health            3799
weather           3298
music             2547
movies            1996
entertainment     1487
kids               299
europe               2
northamerica         1
adexperience         1
Name: count, dtype: int64

In [3]:
df.isnull().sum()

News ID            0
Category           0
Topic              0
Headline           0
News body         58
Title entity       0
Entity content     0
dtype: int64

In [4]:
# Fill NaN values to avoid errors
df['Headline'] = df.get('Headline', '').fillna('').astype(str)
df['News body'] = df.get('News body', '').fillna('').astype(str)

In [5]:
list(df.columns)

['News ID',
 'Category',
 'Topic',
 'Headline',
 'News body',
 'Title entity',
 'Entity content']

In [6]:
df.isnull().sum()

News ID           0
Category          0
Topic             0
Headline          0
News body         0
Title entity      0
Entity content    0
dtype: int64

In [7]:
print(df.shape)

(113762, 7)


In [8]:
# Combine Headline + News body
df['text'] = (df['Headline'].str.strip() + ' ' + df['News body'].str.strip()).str.strip()
df['text'] = df['text'].fillna('')

# Feature and target
X = df["text"]
y = df["Category"]

In [ ]:
for i in range(5):
    print("TEXT:", X.iloc[i])
    print("CATEGORY:", y.iloc[i])
    print("-" * 50)


In [9]:
# keep only required columns
df = df[["Headline", "Title entity", "Entity content"]]

In [10]:
# Take only 50% of the data to reduce computation
df = df.sample(frac=0.5, random_state=42).reset_index(drop=True)

In [11]:
df.head()

,Headline,Title entity,Entity content
0,Stars who came out,{},{}
1,19 Ice Cream Pies You'll Want to Make All Summ...,{},{}
2,Mixed-used development will bring variety to d...,{},{}
3,Paul Manafort Seemed Headed to Rikers. Then th...,{'Justice Department': 'United States Departme...,{'United States Department of Justice': {'type...
4,Sean Shelby's Shoes: What's next for Junior Do...,"{'Junior Dos Santos': 'Junior dos Santos', 'UF...","{'Junior dos Santos': {'type': 'item', 'id': '..."


In [12]:
import ast

def parse_dict(x):
    try:
        return ast.literal_eval(x)
    except:
        return {}

df["title_entity_dict"] = df["Title entity"].apply(parse_dict)


In [13]:
import re

def clean_text_ner(text):
    text = re.sub(r"<.*?>", "", str(text))
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

df["clean_text"] = df["Headline"].apply(clean_text_ner)


In [14]:
def tokenize(text):
    return text.split()

df["tokens"] = df["clean_text"].apply(tokenize)


In [16]:
import spacy

In [17]:
nlp = spacy.load("en_core_web_sm")

df["Headline"] = df["Headline"].astype(str)
df["Title entity"] = df["Title entity"].astype(str)

COUNTRIES = ["United States", "India", "Brazil", "China", "Mexico", "Canada"]
PERSON_PATTERN = r"^[A-Z][a-z]+(\s[A-Z][a-z]+)+$"
ORG_KEYWORDS = ["Corporation", "Authority", "Committee", "Association", "University", "Agency", "Company", "FC", "Ltd"]


def infer_entity_type(expanded):
    expanded = expanded.strip()

    if re.match(PERSON_PATTERN, expanded):
        return "PERSON"

    if expanded in COUNTRIES:
        return "LOCATION"

    if any(k in expanded for k in ORG_KEYWORDS):
        return "ORG"

    return "MISC"

def convert_to_bio(text, entity_string):
    tokens = text.split()
    tags = ["O"] * len(tokens)

    if entity_string == "{}":
        return tokens, tags

    try:
        ent_dict = ast.literal_eval(entity_string)
    except:
        return tokens, tags

    lower_tokens = [w.lower().strip(".,!?") for w in tokens]

    for surface, expanded in ent_dict.items():
        clean_surface = surface.replace("'s", "").strip()
        stoks = clean_surface.split()
        stoks = [w.lower().strip(".,!?") for w in stoks]
        n = len(stoks)

        ent_type = infer_entity_type(expanded)

        # Search entity span safely
        for i in range(len(tokens)):
            try:
                if lower_tokens[i:i+n] == stoks:
                    tags[i] = f"B-{ent_type}"
                    for j in range(i+1, i+n):
                        if j < len(tags):   # SAFETY CHECK
                            tags[j] = f"I-{ent_type}"
            except:
                continue

    return tokens, tags



sentences = []
labels = []

for _, row in df.iterrows():
    s, t = convert_to_bio(row["Headline"], row["Title entity"])
    sentences.append(s)
    labels.append(t)

print("DATA READY — Samples:", len(sentences))


DATA READY — Samples: 56881


In [ ]:
#!pip install keras tensorflow
     

In [25]:
# ENCODING & VOCAB GENERATION

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

word_set = set(w for s in sentences for w in s)
tag_set = set(t for seq in labels for t in seq)

#word2idx = {w: i+2 for i, w in enumerate(sorted(word_set))
word2idx = {w: i for i, w in enumerate(word2idx.keys())}

word2idx[""] = 0
word2idx[""] = 1

tag2idx = {t: i for i, t in enumerate(sorted(tag_set))}
idx2tag = {v: k for k, v in tag2idx.items()}

MAX_LEN = 60

X = [[word2idx.get(w, 1) for w in seq] for seq in sentences]
X = pad_sequences(X, maxlen=MAX_LEN, padding="post")

y = [[tag2idx[t] for t in seq] for seq in labels]
y = pad_sequences(y, maxlen=MAX_LEN, padding="post")
y_cat = to_categorical(y, num_classes=len(tag2idx))

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y_cat, test_size=0.2, random_state=42
)

VOCAB_SIZE = len(word2idx)
NUM_TAGS = len(tag2idx)

print("VOCAB SIZE:", VOCAB_SIZE)
print("NUM TAGS:", NUM_TAGS)

     

VOCAB SIZE: 72461
NUM TAGS: 8


In [27]:
max(word2idx.values()), len(word2idx)


(72459, 72461)

In [28]:

#  Word2vec

import numpy as np
from gensim.models import Word2Vec
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)

embedding_w2v = np.zeros((VOCAB_SIZE, 100))
for w, i in word2idx.items():
    embedding_w2v[i] = w2v_model.wv[w] if w in w2v_model.wv else np.random.normal(0,0.6,100)

Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


In [30]:

# GloVe

GLOVE_PATH = "C:/Users/Malathi M/OneDrive/Documents/MDTE25/guvi final project/Main project/NER/glove.6B.100d.txt"
import numpy as np

glove_index = {}
with open(GLOVE_PATH, encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype="float32")
        glove_index[word] = vector

print("Total embeddings found:", len(glove_index))

# Matrix

embedding_glove = np.zeros((VOCAB_SIZE, 100))

for w, i in word2idx.items():
    embedding_glove[i] = glove_index.get(
        w, np.random.normal(scale=0.6, size=(100,))
    )


Total embeddings found: 400000


Rule-based / Heuristic Baseline:

In [53]:
import re


In [54]:
PATTERNS = {
    "PERSON": re.compile(r"\b([A-Z][a-z]+(?:\s[A-Z][a-z]+)*)\b"),
    
    "DATE": re.compile(
        r"\b(\d{1,2}[/-]\d{1,2}[/-]\d{2,4}|"      # 12/05/2023
        r"\d{4}|"                                # 2023
        r"(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s\d{1,2},?\s\d{4})\b",
        re.IGNORECASE
    ),

    "ORG": re.compile(
        r"\b([A-Z][a-zA-Z& ]+(?:Inc|Ltd|LLC|Corp|University|Institute))\b"
    ),

    "LOC": re.compile(
        r"\b(New York|London|Paris|India|USA|Germany|China)\b"
    )
}


In [55]:
def rule_based_ner(text):
    entities = []

    for label, pattern in PATTERNS.items():
        for match in pattern.finditer(text):
            entities.append({
                "text": match.group(),
                "label": label,
                "start": match.start(),
                "end": match.end()
            })

    return entities


In [56]:
text = "Barack Obama visited New York on July 4, 2012 and spoke at Harvard University."

entities = rule_based_ner(text)

for e in entities:
    print(e)


{'text': 'Barack Obama', 'label': 'PERSON', 'start': 0, 'end': 12}
{'text': 'New York', 'label': 'PERSON', 'start': 21, 'end': 29}
{'text': 'July', 'label': 'PERSON', 'start': 33, 'end': 37}
{'text': 'Harvard University', 'label': 'PERSON', 'start': 59, 'end': 77}
{'text': 'July 4, 2012', 'label': 'DATE', 'start': 33, 'end': 45}
{'text': 'Harvard University', 'label': 'ORG', 'start': 59, 'end': 77}
{'text': 'New York', 'label': 'LOC', 'start': 21, 'end': 29}


In [57]:
def ner_to_bio(tokens, entities):
    tags = ["O"] * len(tokens)

    for ent in entities:
        ent_tokens = ent["text"].split()
        for i in range(len(tokens)):
            if tokens[i:i+len(ent_tokens)] == ent_tokens:
                tags[i] = f"B-{ent['label']}"
                for j in range(1, len(ent_tokens)):
                    tags[i+j] = f"I-{ent['label']}"

    return list(zip(tokens, tags))


In [58]:
tokens = text.split()
bio_tags = ner_to_bio(tokens, entities)
bio_tags


[('Barack', 'B-PERSON'),
 ('Obama', 'I-PERSON'),
 ('visited', 'O'),
 ('New', 'B-LOC'),
 ('York', 'I-LOC'),
 ('on', 'O'),
 ('July', 'B-DATE'),
 ('4,', 'I-DATE'),
 ('2012', 'I-DATE'),
 ('and', 'O'),
 ('spoke', 'O'),
 ('at', 'O'),
 ('Harvard', 'O'),
 ('University.', 'O')]

BILSTM,BILSTM-CRF  DL MODEL

In [ ]:

# !pip uninstall -y tensorflow keras
# !pip install tensorflow==2.11 keras==2.11 tensorflow-addons==0.20.0
# !pip install keras-crf

In [ ]:

# !pip install tensorflow-addons==0.22.0

In [31]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, TimeDistributed, Dense
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)

# ============================================
# Regular BiLSTM Model
# ============================================

def build_bilstm(embedding_matrix):
    inp = Input(shape=(MAX_LEN,))
    emb = Embedding(
        VOCAB_SIZE, 100,
        weights=[embedding_matrix],
        mask_zero=True,
        trainable=False
    )(inp)

    x = Bidirectional(LSTM(128, return_sequences=True))(emb)
    out = TimeDistributed(Dense(NUM_TAGS, activation="softmax"))(x)

    model = Model(inp, out)
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


# ============================================
# BiLSTM + CRF-like loss
# (NO tensorflow_addons needed)
# ============================================

def build_bilstm_crf(embedding_matrix):
    inp = Input(shape=(MAX_LEN,))
    emb = Embedding(
        VOCAB_SIZE, 100,
        weights=[embedding_matrix],
        mask_zero=True,
        trainable=False
    )(inp)

    x = Bidirectional(LSTM(128, return_sequences=True))(emb)
    logits = TimeDistributed(Dense(NUM_TAGS))(x)

    # Learnable CRF transition matrix
    transitions = tf.Variable(
        tf.random.uniform(shape=(NUM_TAGS, NUM_TAGS)),
        name="transition_matrix"
    )

    def crf_loss(y_true, y_pred):
        """
        y_true → one-hot target
        y_pred → logits
        """
        y_true_idx = tf.argmax(y_true, axis=-1)  # (batch, seq)

        # emission log probabilities
        log_softmax = tf.nn.log_softmax(y_pred, axis=-1)

        # likelihood of correct token prediction
        token_ll = tf.reduce_sum(
            tf.reduce_sum(
                tf.one_hot(y_true_idx, NUM_TAGS) * log_softmax,
                axis=-1
            ),
            axis=-1
        )

        # transition score
        seq_score = 0.0

        for t in range(MAX_LEN - 1):
            curr = y_true_idx[:, t]
            nxt = y_true_idx[:, t + 1]

            seq_score += tf.gather_nd(
                transitions,
                tf.stack([curr, nxt], axis=1)
            )

        loss = -(token_ll + seq_score)

        return tf.reduce_mean(loss)

    model = Model(inp, logits)
    model.compile(optimizer="adam", loss=crf_loss)

    return model

In [32]:
results = []
models_dict = {}

print("\nTraining MODEL-1: BiLSTM + Word2Vec")
model_1 = build_bilstm(embedding_w2v)
model_1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32, callbacks=[early_stop])
models_dict["1"] = model_1


Training MODEL-1: BiLSTM + Word2Vec
Epoch 1/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 188s 127ms/step - accuracy: 0.9083 - loss: 0.3610 - val_accuracy: 0.9099 - val_loss: 0.3237
Epoch 2/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 180s 127ms/step - accuracy: 0.9115 - loss: 0.3108 - val_accuracy: 0.9113 - val_loss: 0.3069
Epoch 3/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 171s 120ms/step - accuracy: 0.9134 - loss: 0.2960 - val_accuracy: 0.9132 - val_loss: 0.2989
Epoch 4/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 169s 119ms/step - accuracy: 0.9149 - loss: 0.2855 - val_accuracy: 0.9139 - val_loss: 0.2923
Epoch 5/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 185s 130ms/step - accuracy: 0.9165 - loss: 0.2766 - val_accuracy: 0.9142 - val_loss: 0.2917


In [33]:
print("\nTraining MODEL-2: BiLSTM + GloVe")
model_2 = build_bilstm(embedding_glove)
model_2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32, callbacks=[early_stop])
models_dict["2"] = model_2


Training MODEL-2: BiLSTM + GloVe
Epoch 1/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 181s 124ms/step - accuracy: 0.9086 - loss: 0.3570 - val_accuracy: 0.9148 - val_loss: 0.2987
Epoch 2/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 190s 134ms/step - accuracy: 0.9230 - loss: 0.2604 - val_accuracy: 0.9261 - val_loss: 0.2491
Epoch 3/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 198s 139ms/step - accuracy: 0.9335 - loss: 0.2159 - val_accuracy: 0.9319 - val_loss: 0.2307
Epoch 4/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 176s 124ms/step - accuracy: 0.9416 - loss: 0.1833 - val_accuracy: 0.9330 - val_loss: 0.2223
Epoch 5/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 183s 129ms/step - accuracy: 0.9493 - loss: 0.1556 - val_accuracy: 0.9349 - val_loss: 0.2232


In [36]:
print("\nTraining MODEL-3: BiLSTM-CRF + Word2Vec")
model_3 = build_bilstm_crf(embedding_w2v)
model_3.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32, callbacks=[early_stop])
models_dict["3"] = model_3


Training MODEL-3: BiLSTM-CRF + Word2Vec
Epoch 1/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 364s 247ms/step - loss: 18.7810 - val_loss: -13.2679
Epoch 2/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 346s 243ms/step - loss: -24.3470 - val_loss: -30.8737
Epoch 3/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 349s 245ms/step - loss: -33.5180 - val_loss: -35.1961
Epoch 4/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 346s 244ms/step - loss: -36.2035 - val_loss: -36.7597
Epoch 5/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 342s 241ms/step - loss: -37.2827 - val_loss: -37.4397


In [37]:
print("\nTraining MODEL-4: BiLSTM-CRF + GloVe")
model_4 = build_bilstm_crf(embedding_glove)
model_4.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32, callbacks=[early_stop])
models_dict["4"] = model_4


Training MODEL-4: BiLSTM-CRF + GloVe
Epoch 1/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 362s 247ms/step - loss: 54.4582 - val_loss: 22.0937
Epoch 2/5
1422/1422 ━━━━━━━━━━━━━━━━━━━━ 358s 252ms/step - loss: 10.7813 - val_loss: 4.1989


In [38]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

def evaluate_model(model, X_test, y_test):
    y_prob = model.predict(X_test)
    y_pred = (y_prob > 0.5).astype(int)

    precision = precision_score(y_test, y_pred, average="binary")
    recall    = recall_score(y_test, y_pred, average="binary")
    f1        = f1_score(y_test, y_pred, average="binary")

    return precision, recall, f1


In [ ]:
#pip install seqeval


In [44]:
from seqeval.metrics import precision_score, recall_score, f1_score
import numpy as np

def evaluate_ner_model(model, X_val, y_val, idx2tag):
    y_pred = model.predict(X_val)
    
    # If CRF → output already decoded
    if len(y_pred.shape) == 2:
        y_pred_ids = y_pred
    else:
        y_pred_ids = np.argmax(y_pred, axis=-1)

    true_tags = []
    pred_tags = []

    for true_seq, pred_seq in zip(y_val, y_pred_ids):
        true_tags.append([idx2tag[t] for t in true_seq if t != 0])
        pred_tags.append([idx2tag[p] for p in pred_seq if p != 0])

    return (
        precision_score(true_tags, pred_tags),
        recall_score(true_tags, pred_tags),
        f1_score(true_tags, pred_tags)
    )


In [47]:
from seqeval.metrics import precision_score, recall_score, f1_score
import numpy as np

def evaluate_ner_model(model, X_val, y_val, idx2tag):
    # Model predictions
    y_pred = model.predict(X_val)

    # Convert predictions to tag indices
    if len(y_pred.shape) == 2:  # CRF decoded output
        y_pred_ids = y_pred
    else:
        y_pred_ids = np.argmax(y_pred, axis=-1)

    # Convert one-hot labels to indices
    if len(y_val.shape) == 3:  # one-hot
        y_true_ids = np.argmax(y_val, axis=-1)
    else:
        y_true_ids = y_val

    true_tags = []
    pred_tags = []

    for true_seq, pred_seq in zip(y_true_ids, y_pred_ids):
        t_tags = []
        p_tags = []

        for t, p in zip(true_seq, pred_seq):
            if t == 0:  # skip PAD
                continue
            t_tags.append(idx2tag[int(t)])
            p_tags.append(idx2tag[int(p)])

        true_tags.append(t_tags)
        pred_tags.append(p_tags)

    return (
        precision_score(true_tags, pred_tags),
        recall_score(true_tags, pred_tags),
        f1_score(true_tags, pred_tags)
    )


In [48]:
results = []

model_info = {
    "1": ("BiLSTM", "Word2Vec"),
    "2": ("BiLSTM", "GloVe"),
    "3": ("BiLSTM-CRF", "Word2Vec"),
    "4": ("BiLSTM-CRF", "GloVe")
}

for key, model in models_dict.items():
    precision, recall, f1 = evaluate_ner_model(
        model, X_val, y_val, idx2tag
    )

    results.append({
        "Model": model_info[key][0],
        "Embedding": model_info[key][1],
        "Precision": round(precision, 4),
        "Recall": round(recall, 4),
        "F1_Score": round(f1, 4)
    })


356/356 ━━━━━━━━━━━━━━━━━━━━ 44s 122ms/step
356/356 ━━━━━━━━━━━━━━━━━━━━ 45s 121ms/step
356/356 ━━━━━━━━━━━━━━━━━━━━ 44s 120ms/step
356/356 ━━━━━━━━━━━━━━━━━━━━ 43s 117ms/step


In [49]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df


,Model,Embedding,Precision,Recall,F1_Score
0,BiLSTM,Word2Vec,0.5760,0.1071,0.1806
1,BiLSTM,GloVe,0.6173,0.4415,0.5148
2,BiLSTM-CRF,Word2Vec,0.5233,0.1684,0.2548
3,BiLSTM-CRF,GloVe,0.5867,0.1574,0.2483


In [50]:
best_row = results_df.loc[results_df["F1_Score"].idxmax()]
best_row


Model        BiLSTM
Embedding     GloVe
Precision    0.6173
Recall       0.4415
F1_Score     0.5148
Name: 1, dtype: object

In [52]:
best_model_key = results_df["F1_Score"].idxmax() + 1
best_model = models_dict[str(best_model_key)]
best_model.save("best_ner_model.keras")
